# Fan Testing with wyze_sdk

In [30]:
import os
import time
from wyze_sdk import Client
from wyze_sdk.errors import WyzeApiError
from wyze_sdk.api.devices import ThermostatsClient 
from wyze_sdk.models.devices import ThermostatFanMode

### Client Connection Setup to Wyze API
This is assuming access/refresh tokens have already been generated

In [ ]:
client = Client(token=os.getenv('WYZE_ACCESS_TOKEN'))

### Getting Devices and their Properties

In [ ]:
try:
    response = client.devices_list()
    for device in client.devices_list():
        print(f"mac: {device.mac}")
        print(f"nickname: {device.nickname}")
        print(f"is_online: {device.is_online}")
        print(f"product model: {device.product.model}")
        print(f"{device}")
except WyzeApiError as e:
    print(f"Got an error: {e}")

Create env list of devices as dict(device.nickname=device.mac) for all devices. Devices can then be referred to by their nicknames (without spaces) 

In [8]:
dlist = dict()
for device in client.devices_list():
    dlist[device.nickname.replace(" ", "")] = device.mac
print(dlist)


{'WyzeCam': '2CAA8E1C8608', 'BackPorchCamera': '2CAA8EE7AE25', 'LivingRoom': 'CO_EA1_31304635103637394a416a4b'}


### Creating and Manipulating Thermostat Fan

In [29]:
thermostat = client.thermostats.info(device_mac=dlist["LivingRoom"])
attributes = thermostat.get_non_null_attributes()
for item in attributes:
    print(item,attributes[item])

auto_comfort Property auto_comfort: True [API value: 1]
auto_switch_mode 1
cooling_setpoint 50
current_scenario ThermostatScenarioType.HOME
fan_mode ThermostatFanMode.AUTO
heating_setpoint 70
humidity 42
mac CO_EA1_31304635103637394a416a4b
nickname Living Room
push_switch 1
ssid Linksys40
system_mode ThermostatSystemMode.OFF
temperature 69.7
type Thermostat
working_state idle


In [34]:
old_fan_setting = thermostat.fan_mode
client.thermostats.set_fan_mode(device_mac=thermostat.mac,device_model=thermostat.product.model,fan_mode=ThermostatFanMode.ON)
print(thermostat.fan_mode)
time.sleep(30)
client.thermostats.set_fan_mode(device_mac=thermostat.mac,device_model=thermostat.product.model,fan_mode=old_fan_setting)
print(thermostat.fan_mode)

ThermostatFanMode.AUTO
ThermostatFanMode.AUTO
